In [ ]:
import os

# import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

#NLP
import string
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')


In [ ]:
os.chdir('res')

In [ ]:
# load dataset
df = pd.read_csv('mtsamples.csv')
df.tail()

In [ ]:
df.isna().sum()

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df = df[df['transcription'].notna()]
df.isna().sum()

In [ ]:
data_categories = df.groupby(df['medical_specialty'])

i = 0
for cat_name, data_category in data_categories:
    i += 1
    print(f'Category_{i} : {cat_name} : {len(data_category)}')

In [ ]:
filtered_data_categories = data_categories.filter(lambda x : x.shape[0] > 50)
data_categories = filtered_data_categories.groupby(df['medical_specialty'])

i = 0
for cat_name, data_category in data_categories:
    i += 1
    print(f'Category_{i} : {cat_name} : {len(data_category)}')

In [ ]:
plt.figure()
sns.countplot(y = 'medical_specialty', data = filtered_data_categories, palette = 'Set2')
plt.show()

In [ ]:
data = filtered_data_categories[['transcription', 'medical_specialty']]
data

In [ ]:
data.isna().sum()

In [ ]:
def clean_text(text):
    text = text.translate(str.maketrans("","",string.punctuation)) # noktalama isaretlerini kaldir.
    
    text1 = "".join([w for w in text if not w.isdigit()])
    
    replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')
    
    text2 = text1.lower()
    text2 = replace_by_space_re.sub("", text2)
    
    return text2

In [ ]:
def lemmatize_text(text):
    wordlist = []
    lemmatizer = WordNetLemmatizer()
    
    sentences = sent_tokenize(text)
    
    initial_sentence = sentences[0:1]
    final_sentence = sentences[len(sentences)-2:len(sentences)-1]
    
    for sentence in initial_sentence:
        words = word_tokenize(sentence)
        for word in words:
            wordlist.append(lemmatizer.lemmatize(word))

    for sentence in final_sentence:
        words = word_tokenize(sentence)
        for word in words:
            wordlist.append(lemmatizer.lemmatize(word))
    return " ".join(wordlist)

In [ ]:
data["transcription2"] = data["transcription"].apply(lemmatize_text)
data["transcription2"] = data["transcription2"].apply(clean_text)

In [ ]:
data